# Imports and Setup
Run the Code Below

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchvision
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch
import time

!git clone https://github.com/cis700/hw1-release.git
!mv hw1-release/dills/* .
!mv hw1-release hw1

from hw1.helper import Logger

device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Cloning into 'hw1-release'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 26 (delta 6), reused 23 (delta 6), pack-reused 0
Unpacking objects: 100% (26/26), done.
cuda:0


## TensorBoard Integration

In [0]:
#! rm -r ./logs
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

## Google Drive Integration
For Google Drive use only

In [2]:
from google.colab import drive
drive.mount('/content/drive')
local_dir= 'drive/My Drive/CIS700-004/project'
%cd 'drive/My Drive/CIS700-004/project'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/CIS700-004/project


# The Code

## LSTM

In [0]:
class LSTM(nn.Module): 
  def __init__(self, input_size, hidden_size, num_layers, bidirectional= False):
    super(LSTM, self).__init__()
    self.hidden_size= hidden_size
    self.num_layers= num_layers
    self.lstm= nn.LSTM(input_size, hidden_size, 
                       num_layers, batch_first= True, bidirectional= bidirectional)
    
    self.fc= nn.Linear(hidden_size, 2)
  def forward(self, x): 
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    
    out, _ = self.lstm(x, (h0, c0))
    out= self.fc(out[:, -1, :]) #hidden state of last time step
    return out

lstm = LSTM(300, 400, 3, False).to(device)

## RunNet

In [0]:
class RunNet():
  def __init__(self, net, train_load, test_load, lr= 0.0001, conv= False):
    self.logger= Logger('./logs')
    self.conv= conv
    self.net= net
    self.train_load= train_load
    self.test_load= test_load
    self.criterion = nn.CrossEntropyLoss()
    self.optimizer= torch.optim.Adam(self.net.parameters(), lr= lr)
  
  def test_accuracy(self):
    test_c= 0
    test_t= 0
    with torch.no_grad(): 
      for i, data in enumerate(self.test_load, 0):
        inputs, labels= data
        inputs, labels= Variable(inputs), Variable(labels)

        inputs= inputs.to(device)
        labels= labels.to(device)
        
        if not self.conv:
          inputs = inputs.view(inputs.shape[0], -1)

        pred= self.net(inputs)
        _, pred= torch.max(pred, 1)
        test_c= test_c + int((pred==labels).sum())
        test_t= test_t + labels.size(0)
      print("Test Acc:", (test_c/test_t))
      return test_c/test_t

    
  def run(self, max_epochs= 200, log_every= 10, verbose= True):
    step= 0
    for epoch in range(max_epochs): 
        if verbose: 
          print("Epoch:", epoch)

        train_c= 0
        train_t= 0

        test_c= 0
        test_t= 0

        for i, data in enumerate(self.train_load, 0):
            inputs, labels= data
            inputs, labels= Variable(inputs), Variable(labels)

            inputs= inputs.to(device)
            labels= labels.to(device)
            
            if not self.conv:  
              inputs = inputs.view(inputs.shape[0], -1)

            #feed inputs into network
            y_pred = self.net(inputs)
            loss= self.criterion(y_pred, labels)

            #zero gradient
            self.optimizer.zero_grad()

            #backprop
            loss.backward()

            #weight update
            self.optimizer.step()

            #after updating... report training accuracy
            pred= self.net(inputs)
            t_loss= self.criterion(pred, labels)
            _, pred= torch.max(pred, 1)

            train_correct= int((pred==labels).sum())
            train_c= train_c + train_correct
            train_t = train_t + labels.size(0)

            if (step % log_every == 0):
              self.logger.scalar_summary("training accuracy", (train_correct/labels.size(0)), step)
              self.logger.scalar_summary("loss", t_loss.item(), step)
              self.logger.writer.flush()
              print("train: %s, loss: %s"%(train_c/train_t, t_loss.item()))
            step= step + 1
        with torch.no_grad():      
            for i, data in enumerate(self.test_load, 0):
                inputs, labels= data
                inputs, labels= Variable(inputs), Variable(labels)

                inputs= inputs.to(device)
                labels= labels.to(device)
                
                if not self.conv:
                  inputs = inputs.view(inputs.shape[0], -1)

                pred= self.net(inputs)
                _, pred= torch.max(pred, 1)
                test_correct= int((pred==labels).sum())
                test_c= test_c + test_correct
                test_t= test_t + labels.size(0)

        if verbose: 
          print("Train Acc:", (train_c/train_t),"Test Acc:", (test_c/test_t), 
                "Loss:", t_loss.item())

# Test Run

In [0]:
class CustomDataset(Dataset):
  '''
  DO NOT EDIT THIS CLASS
  '''
  def __init__(self, X, y):
      self.len = len(X)           
      self.x_data = torch.from_numpy(X).float()
      self.y_data = torch.from_numpy(y).long()

  def __len__(self):
      return self.len

  def __getitem__(self, idx):
      return self.x_data[idx], self.y_data[idx]

In [42]:
x= np.array([np.random.rand(1, 300) for i in range(1000)])
y= np.array([(int)(np.mean(i)/0.5) for i in x])
print(y)
train_data= CustomDataset(x, y)
x2= np.array([np.random.rand(1, 300) for i in range(1000)])
y2= np.array([(int)(np.mean(i)/0.5) for i in x2])
test_data= CustomDataset(x2, y2)

train_load= DataLoader(train_data, batch_size= 100)
test_load= DataLoader(test_data, batch_size= 100)

print(len(train_load.dataset))
print(len(test_load.dataset))

[0 1 1 1 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1
 0 1 0 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1
 0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0
 1 1 1 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 1 1 0 0 0 1 1 0 0 0
 0 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1
 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 0
 1 0 1 1 0 0 1 1 0 0 1 1 0 1 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0 1 1 1 0 0 0
 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0
 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1
 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1 0 1 1 0 1 0 0 0 0 1 1 1
 1 0 0 0 1 1 1 1 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1
 0 1 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1
 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 0 1 1 0 1 0 0
 0 0 0 1 1 0 1 1 0 1 1 0 

In [45]:
run_net= RunNet(lstm, train_load, test_load, conv= True)
run_net.run()

Epoch: 0
train: 0.67, loss: 1.1307047605514526
Train Acc: 0.886 Test Acc: 0.807 Loss: 0.08376891165971756
Epoch: 1
train: 1.0, loss: 0.04930339753627777
Train Acc: 0.965 Test Acc: 0.79 Loss: 0.054295677691698074
Epoch: 2
train: 1.0, loss: 0.06195160001516342
Train Acc: 0.974 Test Acc: 0.785 Loss: 0.06024729087948799
Epoch: 3
train: 0.99, loss: 0.07583547383546829
Train Acc: 0.975 Test Acc: 0.772 Loss: 0.07751547545194626
Epoch: 4
train: 0.99, loss: 0.09026353806257248
Train Acc: 0.975 Test Acc: 0.759 Loss: 0.10137949883937836
Epoch: 5
train: 0.98, loss: 0.09730991721153259
Train Acc: 0.973 Test Acc: 0.752 Loss: 0.12060822546482086
Epoch: 6
train: 0.99, loss: 0.0935828760266304
Train Acc: 0.978 Test Acc: 0.748 Loss: 0.12727224826812744
Epoch: 7
train: 0.99, loss: 0.0858883187174797
Train Acc: 0.976 Test Acc: 0.746 Loss: 0.1253443956375122
Epoch: 8
train: 0.99, loss: 0.0813286080956459
Train Acc: 0.978 Test Acc: 0.747 Loss: 0.1220644935965538
Epoch: 9
train: 1.0, loss: 0.0814573764801025